In [213]:
# DO NOT modify this cell.
filename = "Buchtel.pgm"
verticalSeam2Remove = 10
horizontalSeam2Remove = 8

# Yes, you can create your own test cases and you should. Do it in a new cell.

In [214]:
# File operation functions (Universal)
import numpy as np

def strip_trailing_whitespace(input_file, output_file):
    with open(input_file, 'r') as f_in:
        with open(output_file, 'w') as f_out:
            for line in f_in:
                stripped_line = line.strip()
                f_out.write(stripped_line + '\n')

def calculate_energy(image):

    diff_x = np.abs(np.diff(image, axis=1))
    diff_y = np.abs(np.diff(image, axis=0))

    rows, cols = image.shape[:2]

    x_energy = np.zeros_like(image)
    y_energy = np.zeros_like(image)

    x_energy[:, 0] = diff_x[:, 0] # First column (x=0)
    x_energy[:, cols-1] = diff_x[:, cols-2] # Last column (x=cols-1)
    x_energy[:, 1:cols-1] = diff_x[:, 1:] + diff_x[:, :-1] # Middle columns (1 to cols-2)

    y_energy[0, :] = diff_y[0, :] # First row (y=0)
    y_energy[rows-1, :] = diff_y[rows-2, :] # Last row (y=rows-1)
    y_energy[1:rows-1, :] = diff_y[1:, :] + diff_y[:-1, :] # Middle rows (1 to rows-2)

    return x_energy + y_energy

def calculate_cumulative_energy(energy):

    rows = energy.shape[0]
    cumulative_energy = np.zeros_like(energy)

    # Initialize the cumulative energy with the first row of energy
    cumulative_energy[0, :] = energy[0, :]

    # Calculate cumulative energy for the remaining rows
    for y in range(1, rows):
        # Calculate the minimum cumulative energy based on the diff values
        # Left shift of cumulative_energy[y-1]
        shift_left = np.roll(cumulative_energy[y-1], 1)
        # Right shift of cumulative_energy[y-1]
        shift_right = np.roll(cumulative_energy[y-1], -1)

        # Compute the minimum cumulative energy for each pixel
        min_energy = energy[y] + cumulative_energy[y-1]
        min_energy[1:] = np.minimum(min_energy[1:], energy[y, 1:] + shift_left[1:])
        min_energy[:-1] = np.minimum(min_energy[:-1], energy[y, :-1] + shift_right[:-1])

        # Assign the computed minimum cumulative energy to the current row of cumulative_energy
        cumulative_energy[y] = min_energy

    return cumulative_energy

def find_seam_path(cumulative_energy):
    """Find the seam path with minimum cumulative energy."""
    rows, cols = cumulative_energy.shape[:2]
    min_value_x = np.argmin(cumulative_energy[-1, :])

    seam_path = []
    for y in range(rows - 1, -1, -1):
        seam_path.append((y, min_value_x))

        if y > 0:
            # Determine x indices and values of pixels in the row above
            neighbors = [(min_value_x + dx, cumulative_energy[y - 1, min_value_x + dx] if 0 <= min_value_x + dx < cols else float('inf')) for dx in [-1, 0, 1]]

            # Find the minimum value and corresponding x index
            min_value_x = min(neighbors, key=lambda x: x[1])[0]

    return seam_path

def remove_seam(image, seam_path):
    """Remove pixels along the seam path to create a carved image."""
    rows, cols = image.shape[:2]

    carved_image = np.zeros((rows, cols - 1), dtype=image.dtype)

    for y, x in seam_path:
        carved_image[y, :] = np.delete(image[y, :], x, axis=0)

    return carved_image

def seam_carve(image, r=None, g=None, b=None):
    """Perform seam carving on the image."""

    if r is not None and g is not None and b is not None:
        # Color image (PPM)
        energy_r = calculate_energy(r)
        energy_g = calculate_energy(g)
        energy_b = calculate_energy(b)
        energy = energy_r + energy_g + energy_b
    else:
        # Grayscale image (PGM)
        energy = calculate_energy(image)

    cumulative_energy = calculate_cumulative_energy(energy)

    # Find the seam path with minimum cumulative energy
    seam_path = find_seam_path(cumulative_energy)

    # Remove pixels along the seam path to create a carved image
    if r is not None and g is not None and b is not None:
        carved_image = remove_seam(image, seam_path)
        carved_r = remove_seam(r, seam_path)
        carved_g = remove_seam(g, seam_path)
        carved_b = remove_seam(b, seam_path)
        return carved_image, carved_r, carved_g, carved_b
    else:
        carved_image = remove_seam(image, seam_path)
        return carved_image


In [215]:
# File operation functions (PGM)

import numpy as np
import os

def load_pgm_as_numpy_array(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    file_format = lines[0].strip()
    if file_format != 'P2':
        raise ValueError(f"Unsupported file format: {file_format}")

    width, height = map(int, lines[2].split())
    max_value = int(lines[3])

    pixel_values = []

    for line in lines[4:]:
        pixel_values.extend(map(int, line.split()))

    image_np = np.array(pixel_values).reshape(height, width)

    return image_np, max_value

def save_pgm(image_filepath, image_np, max_value, v, h):
    base_filename, ext = os.path.splitext(os.path.basename(image_filepath))
    output_directory = 'test/experimental/'
    output_filename = f"{output_directory}{base_filename}_processed_{v}_{h}{ext}"

    os.makedirs(output_directory, exist_ok=True)

    with open(output_filename, 'w') as file:
        file.write("P2\n")
        file.write(f"# Created by IrfanView\n")
        file.write(f"{image_np.shape[1]} {image_np.shape[0]}\n")
        file.write(f"{max_value}\n")

        for row in image_np:
            file.write(" ".join(map(str, row)) + " \n")

    print(f"Processed image saved as: {output_filename}")


In [216]:
def process_pgm(image_filepath, vertical_seams, horizontal_seams=0):
    image_np, max_value = load_pgm_as_numpy_array(image_filepath)

    image_np = carve_seams_pgm(image_np, vertical_seams)

    if horizontal_seams > 0:
        image_np = image_np.T
        image_np = carve_seams_pgm(image_np, horizontal_seams)
        image_np = image_np.T

    save_pgm(image_filepath, image_np, max_value, vertical_seams, horizontal_seams)


def carve_seams_pgm(image_np, num_seams):
    for _ in range(num_seams):
        image_np = seam_carve(image_np)
    return image_np


In [217]:
# Part I : vertical seam removal - pgm

# For Testing
image_filepath = './test/raw/CAS.pgm'
vertical_seams = 20

# For Grading
# image_filepath = filename
# vertical_seams = verticalSeam2Remove

# Entry Point for Part I
process_pgm(image_filepath, vertical_seams)


Processed image saved as: test/experimental/CAS_processed_20_0.pgm


In [218]:
# Part II : both vertical and horizontal seams removal - pgm

# For Testing
image_filepath = './test/raw/CAS_2.pgm'
vertical_seams = 100
horizontal_seams = 50

# For Grading
# image_filepath = filename
# vertical_seams = verticalSeam2Remove
# horizontal_seams = horizontalSeam2Remove

# Entry Point for Part II
process_pgm(image_filepath, vertical_seams, horizontal_seams)

Processed image saved as: test/experimental/CAS_2_processed_100_50.pgm


In [219]:
# File operation functions (PPM)

import numpy as np
import os

def load_ppm_as_numpy_array(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Check file format
    file_format = lines[0].strip()
    if file_format != "P3":
        raise ValueError(f"Unsupported file format: {file_format}")

    # Extract image dimensions and max pixel value
    width, height = map(int, lines[2].split())
    max_value = int(lines[3])

    # Parse pixel data
    pixel_data = []
    for line in lines[4:]:
        values = list(map(int, line.split()))
        pixel_data.extend(values)

    # Reshape pixel data into RGB arrays
    pixel_data = np.array(pixel_data).reshape(height, width, 3)

    # Separate R, G, B channels
    r_np = pixel_data[:, :, 0]
    g_np = pixel_data[:, :, 1]
    b_np = pixel_data[:, :, 2]

    return r_np, g_np, b_np, max_value

def save_ppm(image_filepath, r, g, b, max_value, v, h):
    base_filename, ext = os.path.splitext(os.path.basename(image_filepath))
    output_directory = 'test/experimental/'
    output_filename = f"{output_directory}{base_filename}_processed_{v}_{h}{ext}"

    os.makedirs(output_directory, exist_ok=True)

    with open(output_filename, 'w') as file:
        file.write("P3\n")
        file.write(f"# Created by IrfanView\n")
        file.write(f"{r.shape[1]} {r.shape[0]}\n")
        file.write(f"{max_value}\n")

        # Write pixel values
        for row_r, row_g, row_b in zip(r, g, b):
            for pixel_r, pixel_g, pixel_b in zip(row_r, row_g, row_b):
                file.write(f"{pixel_r} {pixel_g} {pixel_b} ")
            file.write("\n")

    print(f"Processed image saved as: {output_filename}")


In [220]:
def process_ppm(image_filepath, vertical_seams, horizontal_seams=0):
    r, g, b, max_value = load_ppm_as_numpy_array(image_filepath)

    total = r+g+b

    total, r, g, b = carve_seams_ppm(total, r, g, b, vertical_seams)

    if horizontal_seams > 0:

        total = total.T
        r = r.T
        g = g.T
        b = b.T

        total, r, g, b = carve_seams_ppm(total, r, g, b, horizontal_seams)

        total = total.T
        r = r.T
        g = g.T
        b = b.T

    save_ppm(image_filepath, r, g, b, max_value, vertical_seams, horizontal_seams)


def carve_seams_ppm(total, r, g, b, num_seams):
    for _ in range(num_seams):
        total, r, g, b = seam_carve(total, r, g, b,)
    return total, r, g, b


In [221]:
# Part III : both vertical and horizontal seams removal - ppm

# For Testing
filenameG = "./test/raw/flower.ppm"
vSeam2Remove = 300
hSeam2Remove = 100

# For Grading
# filenameG = "flower.ppm"
# vSeam2Remove = 20
# hSeam2Remove = 25

# Entry Point for Part III
process_ppm(filenameG, vSeam2Remove, hSeam2Remove)

Processed image saved as: test/experimental/flower_processed_300_100.ppm


In [222]:
# Test cell
import filecmp
import os

def test_and_compare(input_files, correct_files):
    results = []

    for my_file, correct_file in zip(input_files, correct_files):
        my_file_stripped = add_suffix_to_filename(my_file, '_stripped')
        correct_file_stripped = add_suffix_to_filename(correct_file, '_stripped')

        strip_trailing_whitespace(my_file, my_file_stripped)
        strip_trailing_whitespace(correct_file, correct_file_stripped)

        same = filecmp.cmp(my_file_stripped, correct_file_stripped, shallow=False)
        results.append("Same" if same else "DIFFERENT")

    return results

def add_suffix_to_filename(filename, suffix):
    base, ext = os.path.splitext(filename)
    return base + suffix + ext

# Define test cases
input_files = [
    "./test/experimental/CAS_processed_20_0.pgm",
    "./test/experimental/CAS_2_processed_100_50.pgm",
    "./test/experimental/flower_processed_300_100.ppm"
]

correct_files = [
    "./test/expected/CAS_processed_20_0.pgm",
    "./test/expected/CAS_2_processed_100_50.pgm",
    "./test/expected/flower_300_100.ppm"
]

# Run tests and display results
results = test_and_compare(input_files, correct_files)

for idx, result in enumerate(results, 1):
    print(f"Testing part {idx}:")
    print(result)

Testing part 1:
DIFFERENT
Testing part 2:
DIFFERENT
Testing part 3:
Same
